In [1]:
import os
import numpy as np
from PIL import Image

In [2]:
input_image_1 = Image.open("images/cat_2.jpg").convert("L")
input_image_2 = Image.open("images/tesla_8_bit.png").convert("L")

np_image_1 = np.array(input_image_1)
np_image_2 = np.array(input_image_2)

print("Image size: ", input_image_1.size)
print("Image mode: ", input_image_1.mode)

print("Image size: ", input_image_2.size)
print("Image mode: ", input_image_2.mode)

Image size:  (1280, 853)
Image mode:  L
Image size:  (800, 1072)
Image mode:  L


In [3]:
def add_gaussian_noise(np_image, mean=0, std=25):
    noise = np.random.normal(mean, std, np_image.shape)
    noisy_image = np_image + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    pil_image = Image.fromarray(noisy_image)
    noise_variance = np.var(noise)
    return pil_image, noise_variance

In [4]:
def generate_noisy_images(np_image, num_images, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    variances = []
    for i in range(num_images):
        noisy_image, noise_variance = add_gaussian_noise(np_image)
        noisy_image.save(f"{output_dir}/noisy_image_{i}.png")
        variances.append(noise_variance)

    return variances

In [5]:
def create_average_images(output_dir, image_count_list):
    all_images = [
        np.array(Image.open(os.path.join(output_dir, f"noisy_image_{i+1}.png")), dtype=np.float32)
        for i in range(max(image_count_list))
    ]
    
    average_images = []
    for count in image_count_list:
        avg_image = np.mean(all_images[:count], axis=0)
        average_images.append(Image.fromarray(np.clip(avg_image, 0, 255).astype(np.uint8)))
    return average_images

In [6]:
def calculate_psnr(original_image, noisy_image):
    mse = np.mean((original_image - noisy_image) ** 2)
    psnr = 10 * np.log10(255**2 / mse)
    return psnr

In [7]:
variances_cat = generate_noisy_images(np_image_1, 1000, "outputs/cat")
variances_tesla = generate_noisy_images(np_image_2, 1000, "outputs/tesla")

#save variances in txt file
with open("outputs/cat/variances.txt", "w") as f:
    for variance in variances_cat:
        f.write(f"{variance}\n")

with open("outputs/tesla/variances.txt", "w") as f:
    for variance in variances_tesla:
        f.write(f"{variance}\n")

In [8]:

image_count_list = [9, 99, 999]
average_images_cat = create_average_images("outputs/cat", image_count_list)
average_images_tesla = create_average_images("outputs/tesla", image_count_list)

for i, avg_image in enumerate(average_images_cat):
    avg_image.save(f"outputs/cat/average_image_{image_count_list[i]}.png")

for i, avg_image in enumerate(average_images_tesla):
    avg_image.save(f"outputs/tesla/average_image_{image_count_list[i]}.png")

psnr_cat = [calculate_psnr(np_image_1, np.array(avg_image)) for avg_image in average_images_cat]
psnr_tesla = [calculate_psnr(np_image_2, np.array(avg_image)) for avg_image in average_images_tesla]

print("PSNR for cat image: ", psnr_cat)
print("PSNR for tesla image: ", psnr_tesla)

PSNR for cat image:  [np.float64(31.08771474852093), np.float64(39.16289680669918), np.float64(44.19939942213951)]
PSNR for tesla image:  [np.float64(31.098169312878316), np.float64(38.74673074891382), np.float64(42.833900569860646)]
